In [1]:
import numpy as np
import sympy as sp
from sympy.printing import fcode, numpy
import matplotlib.pyplot as plt

%matplotlib widget

In [2]:
theta, phi = sp.symbols('theta, phi', real=True)
t, r = sp.symbols('t, r', real=True)

### Derivatives
###### Define a position vector in sphericals

In [3]:
r = sp.Function('r')(t)
theta = sp.Function('theta')(t)
phi = sp.Function('phi')(t)

def pos_in_sph(r,theta, phi):
    return sp.Matrix([
    [r*sp.sin(theta)*sp.cos(phi)],
    [r*sp.sin(theta)*sp.sin(phi)],
    [r*sp.cos(theta)],
])

P0 = pos_in_sph(r, theta, phi)

v0_sph = sp.simplify(sp.diff(P0, t))

##### Print it in a way that we can easily turn into code

In [4]:
printer=numpy.NumPyPrinter()
print('Rotation:')
for v in v0_sph: print(printer.doprint(v).replace('numpy', 'np').replace('Derivative', 'd/dt'))


Rotation:
  # Not supported in Python with np:
  # d/dt
  # d/dt
  # d/dt
  # phi
  # r
  # theta
-r(t)*np.sin(phi(t))*np.sin(theta(t))*d/dt(phi(t), t) + r(t)*np.cos(phi(t))*np.cos(theta(t))*d/dt(theta(t), t) + np.sin(theta(t))*np.cos(phi(t))*d/dt(r(t), t)
  # Not supported in Python with np:
  # d/dt
  # d/dt
  # d/dt
  # phi
  # r
  # theta
r(t)*np.sin(phi(t))*np.cos(theta(t))*d/dt(theta(t), t) + r(t)*np.sin(theta(t))*np.cos(phi(t))*d/dt(phi(t), t) + np.sin(phi(t))*np.sin(theta(t))*d/dt(r(t), t)
  # Not supported in Python with np:
  # d/dt
  # d/dt
  # r
  # theta
-r(t)*np.sin(theta(t))*d/dt(theta(t), t) + np.cos(theta(t))*d/dt(r(t), t)


In [5]:
t = sp.symbols('t')
x = sp.Function('x')(t)
y = sp.Function('y')(t)
z = sp.Function('z')(t)

def pos_in_car(x, y, z):
    return sp.Matrix([
    [sp.sqrt(x**2+y**2+z**2)],
    [sp.atan(z/(sp.sqrt(x**2+y**2)))],
    [sp.atan(y/x)],
])
    


P0_sph = pos_in_car(x, y, z)
v0_car = sp.simplify(sp.diff(P0_sph, t))

In [6]:
#printer=numpy.NumPyPrinter()
print('Rotation:')
for v in v0_car: print(printer.doprint(v).replace('numpy', 'np').replace('Derivative', 'd/dt'))

Rotation:
  # Not supported in Python with np:
  # d/dt
  # d/dt
  # d/dt
  # x
  # y
  # z
(x(t)*d/dt(x(t), t) + y(t)*d/dt(y(t), t) + z(t)*d/dt(z(t), t))/np.sqrt(x(t)**2 + y(t)**2 + z(t)**2)
  # Not supported in Python with np:
  # d/dt
  # d/dt
  # d/dt
  # x
  # y
  # z
(-(x(t)*d/dt(x(t), t) + y(t)*d/dt(y(t), t))*z(t) + (x(t)**2 + y(t)**2)*d/dt(z(t), t))/(np.sqrt(x(t)**2 + y(t)**2)*(x(t)**2 + y(t)**2 + z(t)**2))
  # Not supported in Python with np:
  # d/dt
  # d/dt
  # x
  # y
(x(t)*d/dt(y(t), t) - y(t)*d/dt(x(t), t))/(x(t)**2 + y(t)**2)


## Transformation

In [7]:
def warp(x, y, z, theta, phi):
    xprime = x * np.cos(phi) - y * np.sin(phi) * np.cos(theta) + z * np.sin(phi) * np.sin(theta)
    yprime = x * np.sin(phi) + y * np.cos(phi) * np.cos(theta) - z * np.cos(phi) * np.sin(theta)
    zprime = y * np.sin(theta) + z * np.cos(theta)
    return xprime, yprime, zprime


def unwarp(x, y, z, theta, phi):
    xprime = x * np.cos(phi) + y * np.sin(phi)
    yprime = -x * np.sin(phi) * np.cos(theta) + y * np.cos(phi) * np.cos(theta) + z * np.sin(theta)
    zprime = x * np.sin(phi) * np.sin(theta) - y * np.sin(theta) * np.cos(phi) + z * np.cos(theta)
    return xprime, yprime, zprime

def logistic(a, r, r0, dr):
    return np.radians( a / (1 + np.exp((r - r0) / (0.1 * dr))))

In [8]:
def vel_sph_to_car(THETAU, PHIU, VRU, VTU, VPU):
    
    vx = VRU * np.sin(THETAU) * np.cos(PHIU) + VTU * np.cos(THETAU) * np.cos(PHIU) - VPU * np.sin(PHIU)
    vy = VRU * np.sin(THETAU) * np.sin(PHIU) + VTU * np.cos(THETAU) * np.sin(PHIU) + VPU * np.cos(PHIU)
    vz = VRU * np.cos(THETAU)                - VTU * np.sin(THETAU)
    
    return vx, vy, vz

def vel_car_to_sph(THETAC, PHIC, VXW, VYW, VZW):
    
    vr =  VXW * np.sin(THETAC) * np.cos(PHIC) + VYW * np.sin(THETAC) * np.sin(PHIC) + VZW * np.cos(THETAC)
    vt =  VXW * np.cos(THETAC) * np.cos(PHIC) + VYW * np.cos(THETAC) * np.sin(PHIC) - VZW * np.sin(THETAC)
    vp = -VXW * np.sin(PHIC)                  + VYW * np.cos(PHIC)
    
    return vr, vt, vp

def vel_car_to_sph_CART(x, y, z, vx, vy, vz):
    
    # Velocities
    rho    = x**2 + y**2
    r      = x**2 + y**2 + z**2 
    
    vr     = (x*vx + y*vy + z*vz)/np.sqrt(r)
    vtheta = (-(x*vx + y*vy)*z + (rho)*vz)/(np.sqrt(rho)*(r))
    vphi   = (x*vy - y*vx)/(rho)
    return vr, vtheta, vphi

In [9]:
from astropy import constants as c

au = c.au.cgs.value
pc = c.pc.cgs.value
M_sun = c.M_sun.cgs.value
L_sun = c.L_sun.cgs.value
R_sun = c.R_sun.cgs.value
Grav = c.G.cgs.value
m_p = c.m_p.cgs.value

In [10]:
incw = 45.
PAw = 90.

In [11]:
nphi = 60
nr = 15
r0 = np.linspace(0.5, 1.5, nr)
phi0 = np.linspace(0, 2 * np.pi, nphi)
R, PHI = np.meshgrid(r0, phi0, indexing='ij')

In [12]:
x = R * np.cos(PHI)
y = R * np.sin(PHI)
z = np.zeros_like(y)

In [13]:
dr = 0.4
r00 = 1 

inc_a     = logistic(incw, r0, r00, dr) ### Specify the r0 and dr in AU 
PA_a      = logistic(PAw, r0, r00, dr)

In [14]:


### WARP POSITIONS
xw, yw, zw = warp(x, y, z, inc_a[:, None], PA_a[:, None])

# Create spherical coordinates
RW = np.sqrt(xw**2 + yw**2 + zw**2)
THETAW = np.pi / 2.0 - np.arctan2(zw, np.sqrt(xw**2 + yw**2))
PHIW = np.arctan2(yw, xw) + np.pi

In [15]:
### UNWARP POSITIONS?
xu, yu, zu = unwarp(xw, yw, zw, inc_a[:, None], PA_a[:, None])


#RU = np.sqrt(xu**2 + yu**2 + zu**2)
#THETAU = np.pi / 2.0 - np.arctan2(zu, np.sqrt(xu**2 + yu**2))
#PHIU = np.arctan2(yu, xu) + np.pi

RU = np.sqrt(xu**2 + yu**2 + zu**2)
THETAU = np.pi / 2.0 - np.arctan2(zu, np.sqrt(xu**2 + yu**2))
PHIU = (np.arctan2(yu, xu) + np.pi)%(2 * np.pi)

In [17]:
vr = np.zeros_like(RU)
vtheta = np.zeros_like(RU)
vphi = np.sqrt(Grav * M_sun / RU)

#vxu, vyu, vzu = vel_sph_to_car(THETAU, PHIU, vr, vtheta, vphi)
vxw, vyw, vzw = warp(vxu, vyu, vzu, inc_a[:, None], PA_a[:, None])
#vxw, vyw, vzw = warp(vxu, vyu, vzu, inc_a[:, None], PA_a[:, None])

NameError: name 'vxu' is not defined

In [ ]:
f = plt.figure(figsize=(14,9))
ax = f.add_subplot(projection='3d')
ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)
ax.set_zlim(-1.2, 1.2)

#for ir in range(nr):
    #ax.plot(x[ir, :], y[ir, :], z[ir,:], 'k')
    #ax.plot(xw[ir, :], yw[ir, :], zw[ir,:], 'r-', lw=1)
    #ax.plot(xu[ir, :], yu[ir, :], zu[ir,:], 'g--', lw=1)
ax.quiver(xw, yw, zw, vxw, vyw, vzw, length=0.05, normalize=True)
ax.plot_wireframe(xw, yw, zw, alpha=0.3, color='black')
ax.set_xlabel("x")
ax.set_ylabel("y")